# **📝 Introduction**

### Objective:
*This notebook aims to perform an ETL (Extract, Transform, Load) process on a traffic dataset in CSV format. The dataset contains various features related to traffic flow, weather, vehicle count, and accident reports.
The goal is to clean, explore, and prepare the data for analysis and modeling*

### **📥 Import Required Libraries**

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### **📂 Load and Preview Raw Data**

In [25]:
# Load data from CSV
df_raw = pd.read_csv(r'D:\Desktop\DS\ds\1-cleaning\traffic accident.csv')  # Replace with your actual file path
print("First 5 rows:")
display(df_raw.head())
print(f"\nData shape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")

First 5 rows:


,Timestamp,Latitude,Longitude,Vehicle_Count,Traffic_Speed_kmh,Road_Occupancy_%,Traffic_Light_State,Weather_Condition,Accident_Report,Sentiment_Score,Ride_Sharing_Demand,Parking_Availability,Emission_Levels_g_km,Energy_Consumption_L_h,Traffic_Condition
0,2024-03-01 00:00:00,40.842275,-73.703149,205,49.893435,82.652780,Yellow,Clear,0,-0.609199,2,45,450.760055,19.574337,High
1,2024-03-01 00:05:00,40.831119,-73.987354,202,22.383965,45.829298,Green,Clear,0,0.965442,16,1,321.800341,5.385554,High
2,2024-03-01 00:10:00,40.819549,-73.732462,252,46.889699,82.772465,Green,Rain,0,0.289660,16,49,231.152655,10.277477,High
3,2024-03-01 00:15:00,40.725849,-73.980134,37,5.730536,37.695567,Red,Fog,0,-0.271965,66,10,410.384292,29.243279,High
4,2024-03-01 00:20:00,40.813265,-73.961631,64,61.348034,22.313358,Red,Snow,0,-0.797606,3,5,364.466342,16.801459,Low



Data shape: 5500 rows × 15 columns


### **🔍 Inspect Columns and Missing Values**

In [26]:
# Check column types and non-null values
print("Column info and data types:")
df_raw.info()

print("\nMissing values per column:")
display(df_raw.isnull().sum().loc[lambda x: x > 0])

Column info and data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               5500 non-null   object 
 1   Latitude                5500 non-null   float64
 2   Longitude               5500 non-null   float64
 3   Vehicle_Count           5500 non-null   int64  
 4   Traffic_Speed_kmh       5478 non-null   float64
 5   Road_Occupancy_%        5500 non-null   float64
 6   Traffic_Light_State     5500 non-null   object 
 7   Weather_Condition       5500 non-null   object 
 8   Accident_Report         5500 non-null   int64  
 9   Sentiment_Score         5450 non-null   float64
 10  Ride_Sharing_Demand     5500 non-null   int64  
 11  Parking_Availability    5500 non-null   int64  
 12  Emission_Levels_g_km    5500 non-null   float64
 13  Energy_Consumption_L_h  5500 non-null   float64
 14  Traffic_Cond

Traffic_Speed_kmh    22
Sentiment_Score      50
dtype: int64

### **🔤 Explore Unique Values in Categorical Columns**


In [27]:
text_cols = [
    'Traffic_Light_State', 'Weather_Condition', 'Traffic_Condition'
]

for col in text_cols:
    print(f"\nUnique values in '{col}':")
    print(df_raw[col].unique())


Unique values in 'Traffic_Light_State':
['Yellow' 'Green' 'Red' 'Unknown' 'Blinking']

Unique values in 'Weather_Condition':
['Clear' 'Rain' 'Fog' 'Snow' 'Storm']

Unique values in 'Traffic_Condition':
['High' 'Low' 'Medium' 'Light' 'Unknown' 'Heavy' 'Standstill' 'Moderate']


### **🔧 Define the transform_data() Function**


In [28]:
import pandas as pd

def transform_data(df):
    """
    Clean and transform traffic data for analysis.
    """

    # 1. Drop duplicate rows
    df = df.drop_duplicates()

    # 2. Parse Timestamp into datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

    # 3. Drop rows missing critical columns
    critical_cols = ['Timestamp', 'Traffic_Speed_kmh', 'Accident_Report']
    df = df.dropna(subset=critical_cols)

    # 4. Convert numeric columns and fill NaNs with mean
    numeric_cols = ['Vehicle_Count', 'Road_Occupancy_%', 'Sentiment_Score',
                    'Ride_Sharing_Demand', 'Parking_Availability',
                    'Emission_Levels_g_km', 'Energy_Consumption_L_h']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col].fillna(df[col].mean(), inplace=True)

    # 5. Ensure numeric columns are within logical bounds
    df = df[df['Traffic_Speed_kmh'] >= 0]
    df = df[df['Vehicle_Count'] >= 0]
    df = df[df['Road_Occupancy_%'].between(0, 100)]
    df = df[df['Emission_Levels_g_km'] >= 0]
    df = df[df['Energy_Consumption_L_h'] >= 0]

    # 6. Standardize text in categorical columns
    text_cols = ['Traffic_Light_State', 'Weather_Condition', 'Traffic_Condition']
    for col in text_cols:
        df[col] = df[col].astype(str).str.strip().str.title()

    # 7. Remove invalid values in categorical columns
    valid_lights = ['Red', 'Yellow', 'Green']
    valid_weather = ['Clear', 'Rain', 'Fog', 'Snow']
    df = df[df['Traffic_Light_State'].isin(valid_lights)]
    df = df[df['Weather_Condition'].isin(valid_weather)]

    # 8. Smooth Traffic_Speed_kmh by bin means (n_bins=8)
    def smooth_by_bin_means(series, n_bins=8):
        bins = pd.cut(series, bins=n_bins)
        return series.groupby(bins).transform('mean')

    df['Speed_Smoothed'] = smooth_by_bin_means(df['Traffic_Speed_kmh'], n_bins=8)

    # 9. Remove outliers in speed using IQR
    Q1, Q3 = df['Traffic_Speed_kmh'].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    valid_speed = df['Traffic_Speed_kmh'].between(Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    df = df[valid_speed].copy()

    return df


### **💾 Define the load_data_to_csv() Function**


In [29]:
def load_data_to_csv(df, output_file):
    """
    Save the cleaned DataFrame to a CSV file.
    """
    df.to_csv(output_file, index=False)
    print(f"[Load] Data saved to {output_file}")

### **🚀 Run the ETL Pipeline**

In [30]:
# File paths (update as needed)
input_file  = r'traffic accident.csv'             # Raw CSV you provided
output_file = r'cleaned_traffic_data.csv'     # Where cleaned data will be stored

# 1. Extract
df_raw = pd.read_csv(input_file)
print(f"[Extract] Loaded {df_raw.shape[0]} rows from {input_file}")

# 2. Transform
df_clean = transform_data(df_raw)
print(f"[Transform] Data cleaned; resulting shape: {df_clean.shape}")

# 3. Load
load_data_to_csv(df_clean, output_file)

[Extract] Loaded 5500 rows from traffic accident.csv
[Transform] Data cleaned; resulting shape: (5000, 16)
[Load] Data saved to cleaned_traffic_data.csv


C:\Users\YB Store\AppData\Local\Temp\ipykernel_12892\1153647287.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\YB Store\AppData\Local\Temp\ipykernel_12892\1153647287.py:48: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return series.groupby(bins).transform('mean')


### **📥 Load and Preview Cleaned Data**

In [31]:
# Load the cleaned data
df_clean = pd.read_csv(r'cleaned_traffic_data.csv')
print("First 5 rows of the cleaned data:")
display(df_clean.head())

print(f"\nCleaned data shape: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")

First 5 rows of the cleaned data:


,Timestamp,Latitude,Longitude,Vehicle_Count,Traffic_Speed_kmh,Road_Occupancy_%,Traffic_Light_State,Weather_Condition,Accident_Report,Sentiment_Score,Ride_Sharing_Demand,Parking_Availability,Emission_Levels_g_km,Energy_Consumption_L_h,Traffic_Condition,Speed_Smoothed
0,2024-03-01 00:00:00,40.842275,-73.703149,205,49.893435,82.652780,Yellow,Clear,0,-0.609199,2,45,450.760055,19.574337,High,47.177010
1,2024-03-01 00:05:00,40.831119,-73.987354,202,22.383965,45.829298,Green,Clear,0,0.965442,16,1,321.800341,5.385554,High,19.019345
2,2024-03-01 00:10:00,40.819549,-73.732462,252,46.889699,82.772465,Green,Rain,0,0.289660,16,49,231.152655,10.277477,High,47.177010
3,2024-03-01 00:15:00,40.725849,-73.980134,37,5.730536,37.695567,Red,Fog,0,-0.271965,66,10,410.384292,29.243279,High,9.552348
4,2024-03-01 00:20:00,40.813265,-73.961631,64,61.348034,22.313358,Red,Snow,0,-0.797606,3,5,364.466342,16.801459,Low,66.097690



Cleaned data shape: 5000 rows × 16 columns


### **ℹ️ Cleaned Data Info & Descriptive Statistics**

In [32]:
print("Data types and non-null counts:")
df_clean.info()

print("\nDescriptive statistics for numeric columns:")
display(df_clean.describe())

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               5000 non-null   object 
 1   Latitude                5000 non-null   float64
 2   Longitude               5000 non-null   float64
 3   Vehicle_Count           5000 non-null   int64  
 4   Traffic_Speed_kmh       5000 non-null   float64
 5   Road_Occupancy_%        5000 non-null   float64
 6   Traffic_Light_State     5000 non-null   object 
 7   Weather_Condition       5000 non-null   object 
 8   Accident_Report         5000 non-null   int64  
 9   Sentiment_Score         5000 non-null   float64
 10  Ride_Sharing_Demand     5000 non-null   int64  
 11  Parking_Availability    5000 non-null   int64  
 12  Emission_Levels_g_km    5000 non-null   float64
 13  Energy_Consumption_L_h  5000 non-null   float64
 14  Traffic_

,Latitude,Longitude,Vehicle_Count,Traffic_Speed_kmh,Road_Occupancy_%,Accident_Report,Sentiment_Score,Ride_Sharing_Demand,Parking_Availability,Emission_Levels_g_km,Energy_Consumption_L_h,Speed_Smoothed
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,40.749645,-73.847433,153.921200,42.111096,54.748397,0.096000,-0.005652,50.487800,24.613000,272.174927,17.343243,42.111096
std,0.086123,0.086204,83.523342,21.707720,26.145238,0.294621,0.583790,28.484426,14.532511,130.086372,7.208277,21.535796
min,40.600016,-73.999987,10.000000,5.002789,10.005031,0.000000,-0.999819,1.000000,0.000000,50.136855,5.003787,9.552348
25%,40.675403,-73.920281,80.000000,23.195752,32.089653,0.000000,-0.500154,26.000000,12.000000,160.564433,11.098761,19.019345
50%,40.748875,-73.846058,156.000000,42.191599,54.657297,0.000000,-0.010564,50.000000,24.000000,272.045513,17.153791,37.914451
75%,40.824735,-73.771685,226.000000,60.751760,77.581720,0.000000,0.500518,75.000000,37.000000,382.242055,23.516595,56.616777
max,40.899972,-73.700159,299.000000,79.997556,99.999729,1.000000,0.999354,99.000000,49.000000,499.922663,29.995416,75.326933


### **🔡 Unique Value Counts in Key Categorical Columns**

In [33]:
key_cats = ['Traffic_Light_State', 'Weather_Condition', 'Traffic_Condition']
for col in key_cats:
    print(f"\nUnique value counts for `{col}`:")
    print(df_clean[col].value_counts())


Unique value counts for `Traffic_Light_State`:
Traffic_Light_State
Yellow    1726
Green     1651
Red       1623
Name: count, dtype: int64

Unique value counts for `Weather_Condition`:
Weather_Condition
Rain     1279
Snow     1257
Fog      1245
Clear    1219
Name: count, dtype: int64

Unique value counts for `Traffic_Condition`:
Traffic_Condition
High      3166
Medium    1475
Low        359
Name: count, dtype: int64
